In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks

os.chdir('./CycleSR/')

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [2]:
!pip install --upgrade pip
!pip uninstall scikit-image
!pip install scikit-image
import skimage
print(skimage.__version__)
# skimage 0.19.0 is compulsory

     |████████████████████████████████| 2.1 MB 8.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Found existing installation: scikit-image 0.18.3
Uninstalling scikit-image-0.18.3:
  Would remove:
    /usr/local/bin/skivi
    /usr/local/lib/python3.7/dist-packages/scikit_image-0.18.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/scikit_image.libs/libgomp-3300acd3.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/skimage/*
Proceed (Y/n)? y
y
  Successfully uninstalled scikit-image-0.18.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 61.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
0.19.2


In [3]:
import argparse
import glob
import random
import os
from PIL import Image
import numpy as np
import time
import datetime
import sys
import torch


from math import sqrt
import numpy as np 
import torch.nn.init as init
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.autograd import Variable
import itertools
import matplotlib.pyplot as plt
import pdb
import skimage.metrics
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import utils
from utils import set_requires_grad, weights_init_normal, ReplayBuffer, LambdaLR, save_sample, eval, eval_6m, train_eval
from dataset import ImageDataset, ImageDataset_6mm
import ssim


In [4]:
def xavier(param):
    init.xavier_uniform(param)

class SingleLayer(nn.Module):
    def __init__(self, inChannels,growthRate):
        super(SingleLayer, self).__init__()
        self.conv =nn.Conv2d(inChannels,growthRate,kernel_size=3,padding=1, bias=True)
    def forward(self, x):
        out = F.relu(self.conv(x))
        out = torch.cat((x, out), 1)
        return out
    
class SingleBlock(nn.Module):
    def __init__(self, inChannels,growthRate,nDenselayer):
        super(SingleBlock, self).__init__()
        self.block= self._make_dense(inChannels,growthRate, nDenselayer)
        
    def _make_dense(self,inChannels,growthRate, nDenselayer):
        layers = []
        for i in range(int(nDenselayer)):
            layers.append(SingleLayer(inChannels,growthRate))
            inChannels += growthRate
        return nn.Sequential(*layers)
                
    def forward(self, x):
        out=self.block(x)
        return out

class Net(nn.Module):
    def __init__(self,inChannels,growthRate,nDenselayer,nBlock):
        super(Net,self).__init__()
        
        self.conv1 = nn.Conv2d(1,growthRate,kernel_size=3, padding=1,bias=True)
        
        inChannels = growthRate
        
        self.denseblock = self._make_block(inChannels,growthRate, nDenselayer,nBlock)
        inChannels +=growthRate* nDenselayer*nBlock
        
        self.Bottleneck = nn.Conv2d(in_channels=inChannels, out_channels=128, kernel_size=1,padding=0, bias=True)
        
        self.convt1 = nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=4, stride=2, padding=1, bias=True)
        
        self.convt2 =nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=4, stride=2, padding=1, bias=True)
        
        self.conv2 =nn.Conv2d(in_channels=128, out_channels=1, kernel_size=3,padding=1, bias=True)
    
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                xavier(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
    
    def _make_block(self, inChannels,growthRate, nDenselayer,nBlock):
        blocks =[]
        for i in range(int(nBlock)):
            blocks.append(SingleBlock(inChannels,growthRate,nDenselayer))
            inChannels += growthRate* nDenselayer
        return nn.Sequential(* blocks)  
    
    def forward(self,x):
        out = F.relu(self.conv1(x))
        out = self.denseblock(out)                                      
        out = self.Bottleneck(out)
        out = self.convt1(out)
        out = self.convt2(out)
                                         
        HR = self.conv2(out)
        return HR

In [38]:
class HARNet(nn.Module):
    def __init__(self):
        super(HARNet, self).__init__()
        self.conv_block = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv1 = nn.Conv2d(1, 128, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv2 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv3 = nn.Conv2d(192, 64, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv4 = nn.Conv2d(256, 64, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv5 = nn.Conv2d(320, 64, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv6 = nn.Conv2d(384, 1, kernel_size=3, stride=1, padding=1, bias=True)
        self.relu = nn.ReLU()


    def forward(self, x):
        
        residual = x
        output = self.relu(self.conv1(x)) # (1, 128, 400, 400)

        block_residual = output # skip connection
        output = self.relu(self.conv2(output)) # (1, 64, 400, 400)
        for _ in range(19):
          output = self.relu(self.conv_block(output)) # (1, 64, 400, 400)
        output = torch.cat((output, block_residual), 1) # (1, 192, 400, 400)

        block_residual = output
        output = self.relu(self.conv3(output)) # (1, 64, 400, 400)
        for _ in range(19):
          output = self.relu(self.conv_block(output)) # (1, 64, 400, 400)
        output = torch.cat((output, block_residual), 1) # (1, 256, 400, 400)

        block_residual = output
        output = self.relu(self.conv4(output)) # (1, 64, 400, 400)
        for _ in range(19):
          output = self.relu(self.conv_block(output)) # (1, 64, 400, 400)
        output = torch.cat((output, block_residual), 1) # (1, 320, 400, 400)

        block_residual = output
        output = self.relu(self.conv5(output)) # (1, 64, 400, 400)
        for _ in range(19):
          output = self.relu(self.conv_block(output)) # (1, 64, 400, 400)
        output = torch.cat((output, block_residual), 1) # (1, 384, 400, 400)

        output = self.conv6(output)

        output += residual
        
        return output

In [39]:
class ImageDataset(Dataset):
    def __init__(self, root, transforms_test=None, unaligned=False, mode='test'):
        self.transformA = transforms.Compose(transforms_test)
        self.transformB = transforms.Compose(transforms_test)

        self.unaligned = unaligned

        self.files_A = sorted(glob.glob(os.path.join(root, '6x6_256/') + '/*.*'))
        self.files_B = sorted(glob.glob(os.path.join(root, '3x3_256/') + '/*.*'))

    def __getitem__(self, index):
        path_A = self.files_A[index % len(self.files_A)]
        img_A = Image.open(path_A).convert('L')
        # print(self.files_A[index % len(self.files_A)])

        item_A = self.transformA(img_A)
        path_B = path_A.replace("_3.", "_6.").replace("6x6", "3x3")

        item_B = self.transformB(Image.open(path_B).convert('L'))

        return {'A': item_A, 'B': item_B}

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

class ImageDataset_6mm(Dataset):
    def __init__(self, root, transforms_A=None, transforms_B=None, unaligned=False, mode='train'):
        self.transformA = transforms.Compose(transforms_A)
        self.transformB = transforms.Compose(transforms_B)

        self.unaligned = unaligned

        self.files_A = sorted(glob.glob(os.path.join(root, 'LR') + '/*.*'))
        # self.files_B = sorted(glob.glob(root + '/*.*'))

    def __getitem__(self, index):
        path_A = self.files_A[index % len(self.files_A)]

        path_B = path_A
        path_B = path_B.replace("_lr.", "_hr.").replace("LR", "HR")

        item_A = self.transformA(Image.open(path_A).convert('L'))
        item_B = self.transformB(Image.open(path_B).convert('L'))

        return {'A': item_A, 'B': item_B}

    def __len__(self):
        return len(self.files_A)


test_path = "./dataset/test/"
transforms_test = [ 
                transforms.ToTensor(),
                transforms.Normalize((0.5), (0.5))
                ]
dataset = ImageDataset(test_path, transforms_test=transforms_test, unaligned=True)
l = len(dataset)
train, valid = torch.utils.data.random_split(dataset=dataset, lengths=[200, l-200])
dataloader = DataLoader(train, batch_size=1, shuffle=True)


###############
transforms_A = [ 
                transforms.ToTensor(),
                transforms.CenterCrop(256),
                transforms.Resize((256, 256), interpolation=Image.BICUBIC),
                transforms.Normalize((0.5), (0.5))
                ]
transforms_B = [ 
                transforms.ToTensor(),
                transforms.Normalize((0.5), (0.5)),
                transforms.CenterCrop(256)]

test_path = "./dataset/evalution_6mm/parts"
test_dataset = ImageDataset_6mm(test_path, transforms_A=transforms_A, transforms_B=transforms_B, unaligned=False)

In [41]:
# model = Net(16,4,8,8)
model = HARNet()
model.apply(weights_init_normal)
criterion = nn.MSELoss()
model = model.cuda()
criterion = criterion.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [55]:
def eval_6m(model, dataset=test_dataset):
    n = len(dataset)
    num, psnr, ssim, mse, nmi= 0, 0, 0, 0, 0
    model.eval()
    for i in range(n):
        img = dataset[i]['A'].unsqueeze(0).cuda()
        gt = dataset[i]['B'].unsqueeze(0).cuda()


        y = model(img)

        yimg = y.cpu().detach().numpy().squeeze(0).squeeze(0)
        
        # print(gt.shape)
        # print("fdsafasd fsdaf")
        gtimg = gt.cpu().detach().numpy().squeeze(0).squeeze(0)
        psnr += (skimage.metrics.peak_signal_noise_ratio(yimg, gtimg,data_range=2))
        ssim += (skimage.metrics.structural_similarity(yimg, gtimg))
        mse += (skimage.metrics.mean_squared_error(yimg, gtimg))
        nmi += (skimage.metrics.normalized_mutual_information(yimg, gtimg))
        num += 1
    print(" PSNR: %.4f SSIM: %.4f MSE: %.4f NMI: %.4f"%(psnr/num, ssim/num, mse/num, nmi/num))

def eval_3m(model, dataset=valid):
    n = len(dataset)
    num, psnr, ssim, mse, nmi= 0, 0, 0, 0, 0
    model.eval()
    for i in range(n):
        img = dataset[i]['A'].unsqueeze(0).cuda()
        gt = dataset[i]['B'].unsqueeze(0).cuda()
        y = img

        y = model(img)

        yimg = y.cpu().detach().numpy().squeeze(0).squeeze(0)
        
        gtimg = gt.cpu().detach().numpy().squeeze(0).squeeze(0)
        # print(yimg.shape, gtimg.shape)
        psnr += (skimage.metrics.peak_signal_noise_ratio(yimg, gtimg,data_range=2))
        ssim += (skimage.metrics.structural_similarity(yimg, gtimg))
        mse += (skimage.metrics.mean_squared_error(yimg, gtimg))
        nmi += (skimage.metrics.normalized_mutual_information(yimg, gtimg))
        num += 1
    print(" PSNR: %.4f SSIM: %.4f MSE: %.4f NMI: %.4f"%(psnr/num, ssim/num, mse/num, nmi/num))

In [44]:
Tensor = torch.cuda.FloatTensor
input_A = Tensor(1, 1, 256, 256)
input_B = Tensor(1, 1, 256, 256)

In [56]:
for epoch in range(50):
    real_out, fake_out = None, None
    eval_3m(model)
    eval_6m(model)
    for i, batch in enumerate(dataloader):
        lr = Variable(input_A.copy_(batch['A']))
        target = Variable(input_B.copy_(batch['B']))
        # print(lr.size(), model(lr).size())
        loss = criterion(model(lr), target)
        optimizer.zero_grad()
        loss.backward() 
        optimizer.step()
    print("------------"+str(epoch+1)+"--------------")
    
        # break


 PSNR: 18.0245 SSIM: 0.4756 MSE: 0.0640 NMI: 1.0623
 PSNR: 17.8572 SSIM: 0.5039 MSE: 0.0673 NMI: 1.0683
------------1--------------
 PSNR: 18.0426 SSIM: 0.4786 MSE: 0.0637 NMI: 1.0627
 PSNR: 17.6719 SSIM: 0.4915 MSE: 0.0703 NMI: 1.0688
------------2--------------
 PSNR: 17.9760 SSIM: 0.4761 MSE: 0.0645 NMI: 1.0629
 PSNR: 17.4394 SSIM: 0.4780 MSE: 0.0742 NMI: 1.0690
------------3--------------
 PSNR: 18.0839 SSIM: 0.4838 MSE: 0.0633 NMI: 1.0629
 PSNR: 18.0076 SSIM: 0.5139 MSE: 0.0650 NMI: 1.0687
------------4--------------
 PSNR: 18.1072 SSIM: 0.4851 MSE: 0.0628 NMI: 1.0630
 PSNR: 17.8152 SSIM: 0.5023 MSE: 0.0681 NMI: 1.0687
------------5--------------
 PSNR: 18.1187 SSIM: 0.4877 MSE: 0.0627 NMI: 1.0630
 PSNR: 17.9612 SSIM: 0.5132 MSE: 0.0658 NMI: 1.0685
------------6--------------
 PSNR: 18.1217 SSIM: 0.4868 MSE: 0.0625 NMI: 1.0631
 PSNR: 17.8351 SSIM: 0.5058 MSE: 0.0678 NMI: 1.0684
------------7--------------
 PSNR: 18.1246 SSIM: 0.4875 MSE: 0.0625 NMI: 1.0632
 PSNR: 17.8241 SSIM: 0.5

KeyboardInterrupt: ignored